<a href="https://colab.research.google.com/github/EnesGokceDS/Amazon_Reviews_NLP_Capstone_Project/blob/master/Data_cleaning_and_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import subprocess as sp
import sys,os,shutil,pickle,time
from os.path import join
# mount Google Drive
from os.path import expanduser
gd_path=join(expanduser("~"),'gd')
if not os.path.isdir(gd_path):
    try:
        # load Google Drive
        from google.colab import drive,files
        drive.mount('/drive')
        sp.call('ln -s /drive/My\ Drive '+gd_path, shell=True)
    except:
        print('unable to find Google Drive Folder')
os.chdir(gd_path+'/MachineLearning/NLP_capstone_data')
os.listdir('.')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


['Reviews.csv']

In [0]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('Reviews.csv')
df.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 10 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Id                      568454 non-null  int64 
 1   ProductId               568454 non-null  object
 2   UserId                  568454 non-null  object
 3   ProfileName             568438 non-null  object
 4   HelpfulnessNumerator    568454 non-null  int64 
 5   HelpfulnessDenominator  568454 non-null  int64 
 6   Score                   568454 non-null  int64 
 7   Time                    568454 non-null  int64 
 8   Summary                 568427 non-null  object
 9   Text                    568454 non-null  object
dtypes: int64(5), object(5)
memory usage: 43.4+ MB


In [5]:
df.describe().round(1)

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,568454.0,568454.0,568454.0,568454.0,5.684540e+05
mean,284227.5,1.7,2.2,4.2,1.296257e+09
std,164098.7,7.6,8.3,1.3,4.804331e+07
min,1.0,0.0,0.0,1.0,9.393408e+08
25%,142114.2,0.0,0.0,4.0,1.271290e+09
50%,284227.5,0.0,1.0,5.0,1.311120e+09
75%,426340.8,2.0,2.0,5.0,1.332720e+09
max,568454.0,866.0,923.0,5.0,1.351210e+09


In [6]:
# Determine how many missing values exist in the collection, in which case you can use .sum() chained onto is.na()
null_values=df.isna().sum()
null_values=pd.DataFrame(null_values,columns=['null'])
sum_tot=len(df)
null_values['percent']=null_values['null']/sum_tot*100
round(null_values,3).sort_values('percent',ascending=False)

,null,percent
Summary,27,0.005
ProfileName,16,0.003
Id,0,0.000
ProductId,0,0.000
UserId,0,0.000
HelpfulnessNumerator,0,0.000
HelpfulnessDenominator,0,0.000
Score,0,0.000
Time,0,0.000
Text,0,0.000


We have small number of missing values. We can drop them completely.

In [7]:
df= df.dropna()
df.shape

(568411, 10)

# Basic Feature Extraction - 1

Normally, I tried to make data cleaning first. Then, I realized that while making data cleaning, I am losing some of characters that can help data cleaning. Therefore, there will be two part of feature extraction. Here, I will extract features that can't be exracted after data cleaning.

### 1) Number of stopwords

In [8]:
!pip install -q wordcloud
import wordcloud
from nltk.corpus import stopwords
import nltk
import string
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [9]:
df['stopwords'] = df['Text'].apply(lambda x: len([x for x in x.split() if x in stop]))
df[['Text','stopwords']].head()

,Text,stopwords
0,I have bought several of the Vitality canned d...,21
1,Product arrived labeled as Jumbo Salted Peanut...,12
2,This is a confection that has been around a fe...,42
3,If you are looking for the secret ingredient i...,15
4,Great taffy at a great price. There was a wid...,12


In [0]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return count

df['punctuation'] = df['Text'].apply(lambda x: count_punct(x))

In [11]:
df[['Text','punctuation']].head()

,Text,punctuation
0,I have bought several of the Vitality canned d...,3
1,Product arrived labeled as Jumbo Salted Peanut...,7
2,This is a confection that has been around a fe...,18
3,If you are looking for the secret ingredient i...,5
4,Great taffy at a great price. There was a wid...,5


### 2) Number of hastag characters

One more interesting feature which we can extract from a review is calculating the number of hashtags or mentions present in it. This also helps in extracting extra information from our text data.

In [12]:
df['hastags'] = df['Text'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
df[['Text','hastags']].head()

,Text,hastags
0,I have bought several of the Vitality canned d...,0
1,Product arrived labeled as Jumbo Salted Peanut...,0
2,This is a confection that has been around a fe...,0
3,If you are looking for the secret ingredient i...,0
4,Great taffy at a great price. There was a wid...,0


In [13]:
df.hastags.loc[df.hastags != 0].count()

1414

### 3) Number of numerics
Calculate the number of numerics which are present in the tweets can be useful. At least, it doesn't hurt to have such a data!

In [14]:
df['numerics'] = df['Text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df[['Text','numerics']].head()

,Text,numerics
0,I have bought several of the Vitality canned d...,0
1,Product arrived labeled as Jumbo Salted Peanut...,0
2,This is a confection that has been around a fe...,0
3,If you are looking for the secret ingredient i...,0
4,Great taffy at a great price. There was a wid...,0


### 4) Number of Uppercase words
Anger or rage is quite often expressed by writing in UPPERCASE words which makes this a necessary operation to identify those words.

In [15]:
df['upper'] = df['Text'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
df[['Text','upper']].head()

,Text,upper
0,I have bought several of the Vitality canned d...,1
1,Product arrived labeled as Jumbo Salted Peanut...,0
2,This is a confection that has been around a fe...,2
3,If you are looking for the secret ingredient i...,4
4,Great taffy at a great price. There was a wid...,0




---



---



---



# **Text cleaning techniques**

### Make all text lower case

The first pre-processing step which we will do is transform our reviews into lower case. This avoids having multiple copies of the same words. For example, while calculating the word count, ‘Analytics’ and ‘analytics’ will be taken as different words.

In [16]:
df['Text'] = df['Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['Text'].head()

0    i have bought several of the vitality canned d...
1    product arrived labeled as jumbo salted peanut...
2    this is a confection that has been around a fe...
3    if you are looking for the secret ingredient i...
4    great taffy at a great price. there was a wide...
Name: Text, dtype: object

### Removing Punctuation

In [17]:
df['Text'] = df['Text'].str.replace('[^\w\s]','')
df['Text'].head()

0    i have bought several of the vitality canned d...
1    product arrived labeled as jumbo salted peanut...
2    this is a confection that has been around a fe...
3    if you are looking for the secret ingredient i...
4    great taffy at a great price there was a wide ...
Name: Text, dtype: object

### Removal of Stop Words

In [18]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['Text'] = df['Text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['Text'].sample(10)

315141    unless affinity dayold used coffee grounds try...
485299    love popchips however like flavor garlic flavo...
291344    tried many hot sauces years tapatio salsa pica...
354799    get diet taste non diet soda enough saidbr use...
201942    grew tulsa bays english muffins around least h...
566118    sceptical stuff great quite peanut butter clos...
550754    long story short tasty purchased number flavor...
357529    careful research best dry food dogs diet chose...
248056    researched many chai tea products deciding one...
19701     really wish toddler liked seems high quality w...
Name: Text, dtype: object

### Remove html tags

In [19]:
from bs4 import BeautifulSoup

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

strip_html_tags('sdasdasdasd http://t.co/7AzE4IoGMe Risk Assessmen ')

# I will come back to this later

'sdasdasdasd http://t.co/7AzE4IoGMe Risk Assessmen '

### Spell Correction

We’ve all seen tweets with a plethora of spelling mistakes. Our timelines are often filled with hastly sent tweets that are barely legible at times.

In that regard, spelling correction is a useful pre-processing step because this also will help us in reducing multiple copies of words. For example, “Analytics” and “analytcs” will be treated as different words even if they are used in the same sense.

To achieve this we will use the textblob library. 

In [20]:
from textblob import TextBlob
df['Text'][:5].apply(lambda x: str(TextBlob(x).correct()))

0    bought several vitality canned dog food produc...
1    product arrived labelled lumbo halted peanutst...
2    connection around centuries light pillow citie...
3    looking secret ingredient robitussin believe f...
4    great staff great price wide assortment mummy ...
Name: Text, dtype: object

In [0]:
# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [22]:
df['Text'] = df.Text.apply(round1)
df.Text

0         bought several vitality canned dog food produc...
1         product arrived labeled jumbo salted peanutsth...
2         confection around centuries light pillowy citr...
3         looking secret ingredient robitussin believe f...
4         great taffy great price wide assortment yummy ...
                                ...                        
568449    great sesame chickenthis good better resturant...
568450    im disappointed flavor chocolate notes especia...
568451    stars small give  one training session tried t...
568452    best treats training rewarding dog good groomi...
568453    satisfied product advertised use cereal raw vi...
Name: Text, Length: 568411, dtype: object

In [0]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [24]:
df['Text'] = df.Text.apply(round2)
df.Text

0         bought several vitality canned dog food produc...
1         product arrived labeled jumbo salted peanutsth...
2         confection around centuries light pillowy citr...
3         looking secret ingredient robitussin believe f...
4         great taffy great price wide assortment yummy ...
                                ...                        
568449    great sesame chickenthis good better resturant...
568450    im disappointed flavor chocolate notes especia...
568451    stars small give  one training session tried t...
568452    best treats training rewarding dog good groomi...
568453    satisfied product advertised use cereal raw vi...
Name: Text, Length: 568411, dtype: object

Let's check whether the frequent words make sense or not

In [25]:
freq = pd.Series(' '.join(df['Text']).split()).value_counts()[:20]
freq

br         264688
like       251864
good       195348
one        172306
taste      166572
great      163560
coffee     160180
product    146439
flavor     142442
tea        133094
love       126635
food       123770
would      123356
get        108169
really     100414
dont        95555
much        91906
also        86099
little      83197
use         82743
dtype: int64

# Basic Feature Extraction - 2

###  Number of Words

In [27]:
df['word_count'] = df['Text'].apply(lambda x: len(str(x).split(" ")))
df[['Text','word_count']].head()

,Text,word_count
0,bought several vitality canned dog food produc...,23
1,product arrived labeled jumbo salted peanutsth...,18
2,confection around centuries light pillowy citr...,40
3,looking secret ingredient robitussin believe f...,18
4,great taffy great price wide assortment yummy ...,13


Again, let's check the data and number of null values

In [28]:
null_values=df.isna().sum()
null_values=pd.DataFrame(null_values,columns=['null'])
sum_tot=len(df)
null_values['percent']=null_values['null']/sum_tot*100
round(null_values,3).sort_values('percent',ascending=False)

,null,percent
Id,0,0.0
ProductId,0,0.0
UserId,0,0.0
ProfileName,0,0.0
HelpfulnessNumerator,0,0.0
HelpfulnessDenominator,0,0.0
Score,0,0.0
Time,0,0.0
Summary,0,0.0
Text,0,0.0


### Number of characters

In [29]:
df['char_count'] = df['Text'].str.len() ## this also includes spaces
df[['Text','char_count']].head()

,Text,char_count
0,bought several vitality canned dog food produc...,164
1,product arrived labeled jumbo salted peanutsth...,136
2,confection around centuries light pillowy citr...,285
3,looking secret ingredient robitussin believe f...,128
4,great taffy great price wide assortment yummy ...,83


### 3) Average Word Length

In [0]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/(len(words)+0.000001))

In [42]:
df['avg_word'] = df['Text'].apply(lambda x: avg_word(x)).round(1)
df[['Text','avg_word']].head()

,Text,avg_word
0,bought several vitality canned dog food produc...,6.2
1,product arrived labeled jumbo salted peanutsth...,6.6
2,confection around centuries light pillowy citr...,6.1
3,looking secret ingredient robitussin believe f...,6.2
4,great taffy great price wide assortment yummy ...,5.5


In [43]:
df.sample(2)

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,stopwords,punctuation,hastags,numerics,upper,word_count,char_count,avg_word
414717,414718,B005VOOLXM,A2GH8LYNLQSE3F,0,0,3,1332460800,This Coffee is Not Bad,package marley coffee contains bunch pods desi...,52,34,0,0,5,74,477,5.5
57501,57502,B000E4C1OK,A3LRHUAHFFJ91P,1,1,5,1227398400,Great Chili,one cincinnati chili mix happy get gourmet del...,26,12,0,0,2,24,179,6.5


**Let's convert the 'time' column to a meaningful format**  

In [0]:
df['Time'] = pd.to_datetime(df['Time'],unit='s')

In [52]:
df.sample(5)

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,stopwords,punctuation,hastags,numerics,upper,word_count,char_count,avg_word,date
357235,357236,B0006I5M2M,ARXU7TM4XXI1D,3,3,5,2012-03-21,No more breast milk!,let start saying tea drinker coffee person any...,61,15,0,1,12,74,445,5.3,2012-03-21
80391,80392,B0009XPD64,AXSG1JKI2MDD2,0,0,3,2012-01-15,"Good food, but not good for my dog",bought food bought several flavors previously ...,13,6,0,0,2,21,146,6.0,2012-01-15
362838,362839,B001PATNEI,A3UAOF1VP5U8IJ,4,4,5,2010-02-08,Delicious!,love thick chewy rich chocolate brownies ones ...,52,25,0,2,0,61,403,5.8,2010-02-08
27220,27221,B0000TL6CC,A2MAYUVBO639EQ,0,4,1,2010-06-15,"Nasty consistency, a lot of money for...?",sausage paste plastic tube shaped like sausage...,101,53,0,0,5,94,620,5.6,2010-06-15
123472,123473,B0006VB3SG,A175QDT8DHNG7A,0,0,5,2012-08-13,Long Lasting,two dogs one regular chewer one powerful chewe...,26,6,0,0,1,35,212,5.1,2012-08-13


In [0]:
df= df.drop('date', axis= 1)

Lastly, we don't need 'ProfileName' feature because we already have 'UserId'. Therefore, we can drop 'ProfileName'

In [0]:
df= df.drop('ProfileName', axis= 1)

In [54]:
list(df)

['Id',
 'ProductId',
 'UserId',
 'HelpfulnessNumerator',
 'HelpfulnessDenominator',
 'Score',
 'Time',
 'Summary',
 'Text',
 'stopwords',
 'punctuation',
 'hastags',
 'numerics',
 'upper',
 'word_count',
 'char_count',
 'avg_word']

In [46]:
df.sample(5)

,Id,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,stopwords,punctuation,hastags,numerics,upper,word_count,char_count,avg_word
216927,216928,B0044X5RWA,A2A98IZ6QUMIUX,40,45,4,1309132800,Amazon price is outrageous!!,bought item local grocery store oz jar would...,59,27,0,2,8,64,403,6.0
267155,267156,B004BKLHOS,AP8D74QD9LUUL,0,0,5,1300320000,tasty AND whole grain!,impressed im one actually likes whole grain pr...,38,16,0,1,2,61,429,6.1
351038,351039,B001P5406O,A2F10B2DYLM1LS,2,3,5,1306713600,"Delicious, Came with a pump",ordered picture one review said came pump send...,26,19,0,1,5,34,194,4.9
277846,277847,B007KPWA8S,A2N6Y6FIANRL3G,3,3,4,1337990400,Good Variety,one advantages keurig k cup type brewers ease ...,87,36,0,10,1,127,722,5.2
231242,231243,B003FDC2I2,A20BF6X7XC8W37,0,0,5,1338508800,Amazing!,looking great tasting readytodrink protein sup...,59,17,0,1,4,60,446,6.6




---
##**Now, let's apply round 1 and round 2 data cleaning processes on 'Summary' column**


Keep in mind that round1 operations make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.

In [55]:
df['Summary'] = df.Summary.apply(round1)
df.Summary

0                      good quality dog food
1                          not as advertised
2                        delight says it all
3                             cough medicine
4                                great taffy
                         ...                
568449                   will not do without
568450                          disappointed
568451              perfect for our maltipoo
568452    favorite training and reward treat
568453                           great honey
Name: Summary, Length: 568411, dtype: object

And, round2 operations get rid of some additional punctuation and non-sensical text that was missed the first time around.

In [56]:
df['Summary'] = df.Summary.apply(round2)
df.Summary

0                      good quality dog food
1                          not as advertised
2                        delight says it all
3                             cough medicine
4                                great taffy
                         ...                
568449                   will not do without
568450                          disappointed
568451              perfect for our maltipoo
568452    favorite training and reward treat
568453                           great honey
Name: Summary, Length: 568411, dtype: object

**Now, let's save this clened processed data as CSV file** 

In [0]:
df.to_csv('Amazon_reviews_processed.csv', index=False)